# Local Electronic Structure and Dynamics of Muon-Polaron Complexes in Fe$_2$O$_3$

## Supplemental material: Candidate muon stopping sites

M. H. Dehn$^{1, 2, 3}$ J. K. Shenton$^{4,*}$ D. J. Arseneau$^3$ W. A. MacFarlane$^{2, 3, 5}$ G.
D. Morris$^3$ A. Maigné$^2$ N. A. Spaldin$^4$, and R. F. Kiefl$^{1, 2, 3}$


$^1$Department of Physics and Astronomy, University of British Columbia, Vancouver, BC V6T 1Z1, Canada    
$^2$Stewart Blusson Quantum Matter Institute, University of British Columbia, Vancouver, BC V6T 1Z4, Canada    
$^3$<span style="font-variant:small-caps;">Triumf</span>, Vancouver, BC V6T 2A3, Canada   
$^4$Department of Materials, ETH Zurich, CH-8093 Zürich, Switzerland   
$^5$Department of Chemistry, University of British Columbia, Vancouver, BC, V6T 1Z1, Canada    




$^*$ For queries about the supplemental information in this notebook contact [J. Kane Shenton](mailto:john.shenton@mat.ethz.ch).

### Summary

In this notebook we summarise the muon candidate stopping sites for three possible overall charge states: +1,0,-1. These correspond to the sites reported in Table 1 of the paper. 

Below we summarise the general computational details used in the paper. We also provide the function we used to parse and extract the local field at the muon site from the output of a [VASP hyperfine tensor calculation](https://www.vasp.at/wiki/index.php/LHYPERFINE). 


### Computational details

* Supercell size: 2x2x2 of rhombohedral cell
    * 80 atoms + 1 'muon'
    * $a = b = c = 10.72184487$  Å
    * $\alpha=\beta=\gamma = 55.113^\circ$
* DFT code: VASP version 5.4.4
* Plane-wave cutoff energy: 700 eV
* $\Gamma$-centred 4x4x4 k-point mesh was used for relaxations and 8x8x8 for DOS calculations.
* Exchange-correlation functional: LDA
* $\mathrm{U_{eff}} = 4 $ eV Hubbard correction (Dudarev scheme)
* SCF energy tolerance: 1E-7 eV
* Maximum force tolerance: 5 meV/Å
* The following PAWs were used:
    * Fe_pv 02Aug2007
    * O 22Mar2012
    * H 06May1998

The full `INCAR`, `KPOINTS`, `POSCAR`, `CONTCAR` and `OUTCAR` files can be found in the subdirectories.

### Setup

In [1]:
# ASE version 3.19.1
from ase.io import read

# Numpy version 1.16.4
import numpy as np

# re version: 2.2.1
import re

In order to reduce the size of this repository, we have compressed all of the files used for this analysis. In order to compress or decompress the files, run one of the cells below:

In [15]:
# --- Compress --- #
!gzip -r muon_sites/

In [ ]:
# --- De-compress --- #
!gzip -dr muon_sites/

We also need a function to parse the hyperfine tensor [as calculated by VASP](https://www.vasp.at/wiki/index.php/LHYPERFINE). We need to scale this by the gyromagnetic ratio of the muon and a factor of 1/2 in order to obtain the effective field and hence the predicted precession frequency for the muon.

In [2]:
def get_hyperfine(outcar, natoms, muon_index=-1):
    """
    Function to parse the vasp hyperfine tensor of the 'muon' from an OUTCAR file (final ionic step).
    
    Inputs:
        outcar (string): path to OUTCAR file
        natoms (int): total number of atoms in the cell
        muon_index (int): python index of the muon (typically the last atom in the POSCAR so defaults to -1)
    
    It returns the 
    """
    

    # muon gyromagnetic ratio:
    gamma_mu = 851.616 / (2*np.pi) # MHz /T

    re_totalspin  = re.compile("Total magnetic moment S")

    re_contact  = re.compile("Fermi contact \(isotropic\) hyperfine coupling parameter \(MHz\)")
    re_dipole   = re.compile("Dipolar hyperfine coupling parameters \(MHz\)")

    contacts = np.zeros(natoms)
    dipole_tensors = np.zeros((natoms, 3, 3))

    with open(outcar) as f:
        for line in f:
            if re_totalspin.search(line):
                totalS = float(line.split()[-1])

            if re_contact.search(line):
                next(f) # -------------------------------------------------------------
                next(f) #  ion      A_pw      A_1PS     A_1AE     A_1c      A_tot
                next(f) # -------------------------------------------------------------

                for i in range(natoms):
                    A_tot = float(next(f).split()[-1]) # just take the total contact interaction
                    contacts[i] = A_tot



            if re_dipole.search(line):
                next(f) # -------------------------------------------------------------
                next(f) #    ion      A_xx      A_yy      A_zz      A_xy      A_xz      A_yz
                next(f) # -------------------------------------------------------------

                for i in range(natoms):
                    A_dip = np.array((next(f).split())).astype(float)

                    # diagonal:
                    dipole_tensors[i][0][0] = A_dip[1]
                    dipole_tensors[i][1][1] = A_dip[2]
                    dipole_tensors[i][2][2] = A_dip[3]

                    #off-diagonal
                    dipole_tensors[i][0][1] = A_dip[4]
                    dipole_tensors[i][1][0] = A_dip[4]

                    dipole_tensors[i][0][2] = A_dip[5]
                    dipole_tensors[i][2][0] = A_dip[5]

                    dipole_tensors[i][1][2] = A_dip[6]
                    dipole_tensors[i][2][1] = A_dip[6]

    
    A_c = contacts[muon_index] * gamma_mu
    A_dip = dipole_tensors[muon_index] * gamma_mu
        
    A = A_dip + np.eye(3)*A_c
    
    # B-field
    Bfield = 0.5 * A[2]
    B_magnitude = np.linalg.norm(Bfield)

    # z-direction
    z = np.array([0,0,1])
    # get new theta
    theta = 180*np.arccos(np.array(Bfield).dot(z) / B_magnitude) / np.pi
    if theta > 90:
        theta = 180 - theta

    # Store Results 
    res = {'A_c'         : A_c,
           'A_dip'       : A_dip,
           'A'           : A,
           'Bfield'      : Bfield,
           'B_magnitude' : B_magnitude,
           'theta'       : theta}    
    
    return res
    

In [3]:
def print_res(res):
    """
    Prints out, in a pretty way, the local field at the muon sites.
    
    Inputs: 
    res (dict): dictionary returned from the get_hyperfine() function
    """
    
    print('Fermi contact: {:6.3f} MHz'.format(res["A_c"]))
    print('Anisotropic part of hyperfine tensor (MHz):')
    for row in res["A_dip"]:
        print("{:10.3f} {:10.3f} {:10.3f}".format(*row))
        
    print("\nEffective B field: [{:8.3f}, {:8.3f}, {:8.3f}] MHz".format(*res['Bfield']))
    
    print('\n')
    print(40*"-")
    print("Precession freq.    : {:10.1f} MHz".format(res['B_magnitude']))
    print("Angle wrt c ([0001]): {:10.1f} deg".format(res['theta']))
    print(40*"-")

### Bare muon (charge +1 state)

In [6]:
outcar = "./muon_sites/mu_plus/C/20200403-2032-OUTCAR"
mu_plus_C     = read(outcar, format='vasp-out')
mu_plus_C_hf  = get_hyperfine(outcar, natoms= len(mu_plus_C))
print_res(mu_plus_C_hf)

Fermi contact: -1.355 MHz
Anisotropic part of hyperfine tensor (MHz):
   283.547    -28.734    -15.587
   -28.734    167.119     58.553
   -15.587     58.553   -450.667

Effective B field: [  -7.793,   29.276, -226.011] MHz


----------------------------------------
Precession freq.    :      228.0 MHz
Angle wrt c ([0001]):        7.6 deg
----------------------------------------


### Muon-polaron complexes (charge neutral state)

#### C1

In [7]:
outcar = "./muon_sites/mu_zero/C1/20200405-1936-OUTCAR"
mu_zero_C1     = read(outcar, format='vasp-out')
mu_zero_C1_hf  = get_hyperfine(outcar, natoms= len(mu_zero_C1))
print_res(mu_zero_C1_hf)

Fermi contact:  2.846 MHz
Anisotropic part of hyperfine tensor (MHz):
   275.144    -17.485    -26.430
   -17.485    151.939     58.011
   -26.430     58.011   -427.083

Effective B field: [ -13.215,   29.005, -212.118] MHz


----------------------------------------
Precession freq.    :      214.5 MHz
Angle wrt c ([0001]):        8.5 deg
----------------------------------------


#### C2

In [8]:
outcar = "./muon_sites/mu_zero/C2/20200403-2058-OUTCAR"
mu_zero_C2     = read(outcar, format='vasp-out')
mu_zero_C2_hf  = get_hyperfine(outcar, natoms= len(mu_zero_C2))
print_res(mu_zero_C2_hf)

Fermi contact: -5.150 MHz
Anisotropic part of hyperfine tensor (MHz):
   290.595    -21.009      0.678
   -21.009    152.481     57.062
     0.678     57.062   -442.941

Effective B field: [   0.339,   28.531, -224.046] MHz


----------------------------------------
Precession freq.    :      225.9 MHz
Angle wrt c ([0001]):        7.3 deg
----------------------------------------


#### C3

In [9]:
outcar = "./muon_sites/mu_zero/C3/20200810-1412-OUTCAR"
mu_zero_C3     = read(outcar, format='vasp-out')
mu_zero_C3_hf  = get_hyperfine(outcar, natoms= len(mu_zero_C3))
print_res(mu_zero_C3_hf)

Fermi contact: -27.243 MHz
Anisotropic part of hyperfine tensor (MHz):
   263.081    -39.306    -11.521
   -39.306    185.146     58.011
   -11.521     58.011   -448.092

Effective B field: [  -5.760,   29.005, -237.667] MHz


----------------------------------------
Precession freq.    :      239.5 MHz
Angle wrt c ([0001]):        7.1 deg
----------------------------------------


#### C4

In [10]:
outcar = "./muon_sites/mu_zero/C4/20200403-2102-OUTCAR"
mu_zero_C4     = read(outcar, format='vasp-out')
mu_zero_C4_hf  = get_hyperfine(outcar, natoms= len(mu_zero_C4))
print_res(mu_zero_C4_hf)

Fermi contact: -53.944 MHz
Anisotropic part of hyperfine tensor (MHz):
   261.861    -63.839    -60.586
   -63.839    198.700      4.337
   -60.586      4.337   -460.697

Effective B field: [ -30.293,    2.169, -257.321] MHz


----------------------------------------
Precession freq.    :      259.1 MHz
Angle wrt c ([0001]):        6.7 deg
----------------------------------------


#### Compare total energies of the charge-neutral states

In [11]:
print("Total energies of C2, C3, C4 with respect to C1: ")
for C in [mu_zero_C2, mu_zero_C3, mu_zero_C4]:
    e = C.get_potential_energy() - mu_zero_C1.get_potential_energy()
    print(f"{1000*e:10.1f} meV")

Total energies of C2, C3, C4 with respect to C1: 
      12.5 meV
      37.2 meV
      50.6 meV


### Muon-polaron complexes (charge -1 state)

#### C1

In [12]:
outcar = "./muon_sites/mu_minus/C1/20200404-0150-OUTCAR"
mu_minus_C1     = read(outcar, format='vasp-out')
mu_minus_C1_hf  = get_hyperfine(outcar, natoms= len(mu_minus_C1))
print_res(mu_minus_C1_hf)

Fermi contact: -26.295 MHz
Anisotropic part of hyperfine tensor (MHz):
   252.645    -27.785    -22.635
   -27.785    167.391     57.604
   -22.635     57.604   -420.035

Effective B field: [ -11.317,   28.802, -223.165] MHz


----------------------------------------
Precession freq.    :      225.3 MHz
Angle wrt c ([0001]):        7.9 deg
----------------------------------------


#### C2

In [13]:
outcar = "./muon_sites/mu_minus/C2/20200404-0335-OUTCAR"
mu_minus_C2     = read(outcar, format='vasp-out')
mu_minus_C2_hf  = get_hyperfine(outcar, natoms= len(mu_minus_C2))
print_res(mu_minus_C2_hf)

Fermi contact:  2.982 MHz
Anisotropic part of hyperfine tensor (MHz):
   283.683    -12.605    -11.521
   -12.605    136.488     62.483
   -11.521     62.483   -420.171

Effective B field: [  -5.760,   31.242, -208.594] MHz


----------------------------------------
Precession freq.    :      211.0 MHz
Angle wrt c ([0001]):        8.7 deg
----------------------------------------


#### Compare total energies of the charge -1 states

In [14]:
print("Total energy of C2 with respect to C1: ")
e = mu_minus_C2.get_potential_energy() - mu_minus_C1.get_potential_energy()
print(f"{1000*e:10.1f} meV")

Total energy of C2 with respect to C1: 
       3.7 meV
